In [6]:
import pandas as pd 
import numpy as np
pd.options.mode.chained_assignment = None
regions = pd.read_csv("~/Downloads/Forecast Details - Campaign Program.csv")
pd.DataFrame(regions)
regions['Date Pulled'] = pd.datetime.now().date()
regions.columns = ["Function", "Budget_Name", "Campaign", "Marketing_Activity", 
                 "Marketing_Activity_Type", "Line_Item", "Activity_Description", 
                 "Quarter", "Actual", "Planned", "Date_Pulled"]
regions['Campaign'] = regions['Campaign'].str.slice(10,)
regions['Campaign'] = regions['Campaign'].str.replace('-',' ')
regions['Negative_Planned_Tag'] = 'Negative Planned'
regions['Negative_Planned_Tag'][regions['Planned'] < 0] = 'Negative Planned'
regions['Negative_Planned_Tag'][regions['Planned'] >= 0] = 'Positive Planned'
regions['Negative_Actual_Tag'] = 'Negative Actual'
regions['Negative_Actual_Tag'][regions['Actual'] < 0] = 'Negative Actual'
regions['Negative_Actual_Tag'][regions['Actual'] >= 0] = 'Positive Actual'
regions['OpenShift_Tag'] = np.where(regions['Activity_Description'].str.contains('openshift|ocp|container|containers',
                                                                           na=False, case=False),'OpenShift', 
                                np.where(regions['Line_Item'].str.contains('openshift|ocp|container|containers',
                                na=False, case=False), 'OpenShift','Middleware'))
duplicateregions = regions[regions.duplicated(['Function', 'Budget_Name', 'Campaign', 'Marketing_Activity', 
                 'Marketing_Activity_Type', 'Line_Item', 'Activity_Description'])]
uniqueregions = regions[~regions.duplicated(['Function', 'Budget_Name', 'Campaign', 'Marketing_Activity', 
                 'Marketing_Activity_Type', 'Line_Item', 'Activity_Description'])]
duplicatenotzeroregions = duplicateregions[~((duplicateregions['Actual']==0) & (duplicateregions['Planned']==0))]
uniquenotzeroregions = uniqueregions[~((uniqueregions['Actual']==0) & (uniqueregions['Planned']==0))]
allnotzeroregions = pd.concat([duplicatenotzeroregions, uniquenotzeroregions])
duplicatezeroregions = duplicateregions[(duplicateregions['Actual']==0) & (duplicateregions['Planned']==0)]
uniquezeroregions = uniqueregions[(uniqueregions['Actual']==0) & (uniqueregions['Planned']==0)]
allzeroregions = pd.concat([duplicatezeroregions, uniquezeroregions])
allzeroduplicateregions = allzeroregions[allzeroregions.duplicated(['Function', 'Budget_Name', 'Campaign', 'Marketing_Activity', 
                 'Marketing_Activity_Type', 'Line_Item', 'Activity_Description'])]
allzerouniqueregions = allzeroregions[~allzeroregions.duplicated(['Function', 'Budget_Name', 'Campaign', 'Marketing_Activity', 
                 'Marketing_Activity_Type', 'Line_Item', 'Activity_Description'])]
finalregions = pd.concat([allzerouniqueregions, allnotzeroregions])
with pd.ExcelWriter('Regions Data Python.xlsx', engine='openpyxl', mode='a') as writer:
    finalregions.to_excel(writer, sheet_name=str(pd.datetime.now().date()), index=False)
cnad = pd.read_excel("~/Downloads/532---Campaign---Cloud-Native-App-Dev.xls")
pd.DataFrame(cnad)
cnad = cnad[['EXPENSES', 'ID', 'Q1 Plan', 'Q2 Plan', 'Q3 Plan', 'Q4 Plan', 'FY Plan', 'Q1 Actual', 'Q2 Actual',
            'Q3 Actual', 'Q4 Actual', 'FY Actual', 'Marketing Activity', 'Marketing Activity Type', 'Event Type',
            'Activity Description']]
cnad.columns = ['Line_Item', 'Line_Item_ID', 'Q1_Plan', 'Q2_Plan', 'Q3_Plan', 'Q4_Plan', 'FY_Plan', 'Q1_Actual', 
                'Q2_Actual', 'Q3_Actual', 'Q4_Actual', 'FY_Actual', 'Marketing_Activity', 'Marketing_Activity_Type',
                'Event_Type', 'Activity_Description']
cnad['Date_Pulled'] = pd.datetime.now().date()
cnad['Function'] = 'Global Demand Center'
cnad['Campaign'] = 'Cloud Native App Dev'
cnad['Budget_Name'] = 'GDC Cloud Native App Dev'
cnad['OpenShift_Tag'] = np.where(cnad['Activity_Description'].str.contains('openshift|ocp|container|containers',
                                                                           na=False, case=False),'OpenShift', 
                                np.where(cnad['Line_Item'].str.contains('openshift|ocp|container|containers',
                                na=False, case=False), 'OpenShift','Middleware'))
cnad['Q1_Plan'] = np.where(cnad['Q1_Plan'] > 0, cnad['Q1_Plan'], 0)
cnad['Q1_Actual'] = np.where(cnad['Q1_Actual'] > 0, cnad['Q1_Actual'], 0)
cnad['Q2_Plan'] = np.where(cnad['Q2_Plan'] > 0, cnad['Q2_Plan'], 0)
cnad['Q2_Actual'] = np.where(cnad['Q2_Actual'] > 0, cnad['Q2_Actual'], 0)
cnad['Q3_Plan'] = np.where(cnad['Q3_Plan'] > 0, cnad['Q3_Plan'], 0)
cnad['Q3_Actual'] = np.where(cnad['Q3_Actual'] > 0, cnad['Q3_Actual'], 0)
cnad['Q4_Plan'] = np.where(cnad['Q4_Plan'] > 0, cnad['Q4_Plan'], 0)
cnad['Q4_Actual'] = np.where(cnad['Q4_Actual'] > 0, cnad['Q4_Actual'], 0)
cnad['FY_Plan'] = np.where(cnad['FY_Plan'] > 0, cnad['FY_Plan'], 0)
cnad['FY_Actual'] = np.where(cnad['FY_Actual'] > 0, cnad['FY_Actual'], 0)
cnad['Q2_Rolling_Plan'] = cnad['Q1_Plan']+cnad['Q2_Plan']
cnad['Q2_Rolling_Actual'] = cnad['Q1_Actual']+cnad['Q2_Actual']
cnad['Q3_Rolling_Plan'] = cnad['Q2_Rolling_Plan']+cnad['Q3_Plan']
cnad['Q3_Rolling_Actual'] = cnad['Q2_Rolling_Actual']+cnad['Q3_Actual']
cnad['Q4_Rolling_Plan'] = cnad['Q3_Rolling_Plan']+cnad['Q4_Plan']
cnad['Q4_Rolling_Actual'] = cnad['Q3_Rolling_Actual']+cnad['Q4_Actual']
cnad['Marketing_Activity'][cnad['Marketing_Activity_Type'] == 'Digital Program'] = 'Digital'
cnad['Function'][cnad['Marketing_Activity'] == 'Digital'] = 'GDC Digital'
cnadq1plan = cnad[['Line_Item', 'Line_Item_ID', 'Q1_Plan', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag']]
cnadq1plan['Quarter'] = 'Q1'
cnadq1plan['Planned_Actual'] = 'Planned'
cnadq1plan['Planned'] = cnadq1plan['Q1_Plan']
cnadq1actual = cnad[['Line_Item', 'Line_Item_ID', 'Q1_Actual', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag']]
cnadq1actual['Quarter'] = 'Q1'
cnadq1actual['Planned_Actual'] = 'Actual'
cnadq1actual['Actual'] = cnadq1actual['Q1_Actual']
cnadq2plan = cnad[['Line_Item', 'Line_Item_ID', 'Q2_Plan', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag', 'Q2_Rolling_Plan']]
cnadq2plan['Quarter'] = 'Q2'
cnadq2plan['Planned_Actual'] = 'Planned'
cnadq2plan['Planned'] = cnadq2plan['Q2_Plan']
cnadq2actual = cnad[['Line_Item', 'Line_Item_ID', 'Q2_Actual', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag', 'Q2_Rolling_Actual']]
cnadq2actual['Quarter'] = 'Q2'
cnadq2actual['Planned_Actual'] = 'Actual'
cnadq2actual['Actual'] = cnadq2actual['Q2_Actual']
cnadq3plan = cnad[['Line_Item', 'Line_Item_ID', 'Q3_Plan', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag', 'Q3_Rolling_Plan']]
cnadq3plan['Quarter'] = 'Q3'
cnadq3plan['Planned_Actual'] = 'Planned'
cnadq3plan['Planned'] = cnadq3plan['Q3_Plan']
cnadq3actual = cnad[['Line_Item', 'Line_Item_ID', 'Q3_Actual', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag', 'Q3_Rolling_Actual']]
cnadq3actual['Quarter'] = 'Q3'
cnadq3actual['Planned_Actual'] = 'Actual'
cnadq3actual['Actual'] = cnadq3actual['Q3_Actual']
cnadq4plan = cnad[['Line_Item', 'Line_Item_ID', 'Q4_Plan', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag', 'Q4_Rolling_Plan']]
cnadq4plan['Quarter'] = 'Q4'
cnadq4plan['Planned_Actual'] = 'Planned'
cnadq4plan['Planned'] = cnadq4plan['Q4_Plan']
cnadq4actual = cnad[['Line_Item', 'Line_Item_ID', 'Q4_Actual', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag', 'Q4_Rolling_Actual']]
cnadq4actual['Quarter'] = 'Q4'
cnadq4actual['Planned_Actual'] = 'Actual'
cnadq4actual['Actual'] = cnadq4actual['Q4_Actual']
cnad = pd.concat([cnadq1plan, cnadq1actual, cnadq2plan, cnadq2actual, cnadq3plan, cnadq3actual, cnadq4plan,
                  cnadq4actual], sort=True)
with pd.ExcelWriter('Cloud Native Data Python.xlsx', engine='openpyxl', mode='a') as writer:
    cnad.to_excel(writer, sheet_name=str(pd.datetime.now().date()), index=False)
ai = pd.read_excel("~/Downloads/532---Campaign---Agile-Integration.xls")
pd.DataFrame(ai)
ai = ai[['EXPENSES', 'ID', 'Q1 Plan', 'Q2 Plan', 'Q3 Plan', 'Q4 Plan', 'FY Plan', 'Q1 Actual', 'Q2 Actual',
            'Q3 Actual', 'Q4 Actual', 'FY Actual', 'Marketing Activity', 'Marketing Activity Type', 'Event Type',
            'Activity Description']]
ai.columns = ['Line_Item', 'Line_Item_ID', 'Q1_Plan', 'Q2_Plan', 'Q3_Plan', 'Q4_Plan', 'FY_Plan', 'Q1_Actual', 
                'Q2_Actual', 'Q3_Actual', 'Q4_Actual', 'FY_Actual', 'Marketing_Activity', 'Marketing_Activity_Type',
                'Event_Type', 'Activity_Description']
ai['Date_Pulled'] = pd.datetime.now().date()
ai['Function'] = 'Global Demand Center'
ai['Campaign'] = 'Agile Integration'
ai['Budget_Name'] = 'GDC Agile Integration'
ai['OpenShift_Tag'] = np.where(ai['Activity_Description'].str.contains('openshift|ocp|container|containers',
                                                                           na=False, case=False),'OpenShift', 
                                np.where(ai['Line_Item'].str.contains('openshift|ocp|container|containers',
                                na=False, case=False), 'OpenShift','Middleware'))
ai['Q1_Plan'] = np.where(ai['Q1_Plan'] > 0, ai['Q1_Plan'], 0)
ai['Q1_Actual'] = np.where(ai['Q1_Actual'] > 0, ai['Q1_Actual'], 0)
ai['Q2_Plan'] = np.where(ai['Q2_Plan'] > 0, ai['Q2_Plan'], 0)
ai['Q2_Actual'] = np.where(ai['Q2_Actual'] > 0, ai['Q2_Actual'], 0)
ai['Q3_Plan'] = np.where(ai['Q3_Plan'] > 0, ai['Q3_Plan'], 0)
ai['Q3_Actual'] = np.where(ai['Q3_Actual'] > 0, ai['Q3_Actual'], 0)
ai['Q4_Plan'] = np.where(ai['Q4_Plan'] > 0, ai['Q4_Plan'], 0)
ai['Q4_Actual'] = np.where(ai['Q4_Actual'] > 0, ai['Q4_Actual'], 0)
ai['FY_Plan'] = np.where(ai['FY_Plan'] > 0, ai['FY_Plan'], 0)
ai['FY_Actual'] = np.where(ai['FY_Actual'] > 0, ai['FY_Actual'], 0)
ai['Q2_Rolling_Plan'] = ai['Q1_Plan']+ai['Q2_Plan']
ai['Q2_Rolling_Actual'] = ai['Q1_Actual']+ai['Q2_Actual']
ai['Q3_Rolling_Plan'] = ai['Q2_Rolling_Plan']+ai['Q3_Plan']
ai['Q3_Rolling_Actual'] = ai['Q2_Rolling_Actual']+ai['Q3_Actual']
ai['Q4_Rolling_Plan'] = ai['Q3_Rolling_Plan']+ai['Q4_Plan']
ai['Q4_Rolling_Actual'] = ai['Q3_Rolling_Actual']+ai['Q4_Actual']
ai['Marketing_Activity'][ai['Marketing_Activity_Type'] == 'Digital Program'] = 'Digital'
ai['Function'][ai['Marketing_Activity'] == 'Digital'] = 'GDC Digital'
aiq1plan = ai[['Line_Item', 'Line_Item_ID', 'Q1_Plan', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag']]
aiq1plan['Quarter'] = 'Q1'
aiq1plan['Planned_Actual'] = 'Planned'
aiq1plan['Planned'] = aiq1plan['Q1_Plan']
aiq1actual = ai[['Line_Item', 'Line_Item_ID', 'Q1_Actual', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag']]
aiq1actual['Quarter'] = 'Q1'
aiq1actual['Planned_Actual'] = 'Actual'
aiq1actual['Actual'] = aiq1actual['Q1_Actual']
aiq2plan = ai[['Line_Item', 'Line_Item_ID', 'Q2_Plan', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag', 'Q2_Rolling_Plan']]
aiq2plan['Quarter'] = 'Q2'
aiq2plan['Planned_Actual'] = 'Planned'
aiq2plan['Planned'] = aiq2plan['Q2_Plan']
aiq2actual = ai[['Line_Item', 'Line_Item_ID', 'Q2_Actual', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag', 'Q2_Rolling_Actual']]
aiq2actual['Quarter'] = 'Q2'
aiq2actual['Planned_Actual'] = 'Actual'
aiq2actual['Actual'] = aiq2actual['Q2_Actual']
aiq3plan = ai[['Line_Item', 'Line_Item_ID', 'Q3_Plan', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag', 'Q3_Rolling_Plan']]
aiq3plan['Quarter'] = 'Q3'
aiq3plan['Planned_Actual'] = 'Planned'
aiq3plan['Planned'] = aiq3plan['Q3_Plan']
aiq3actual = ai[['Line_Item', 'Line_Item_ID', 'Q3_Actual', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag', 'Q3_Rolling_Actual']]
aiq3actual['Quarter'] = 'Q3'
aiq3actual['Planned_Actual'] = 'Actual'
aiq3actual['Actual'] = aiq3actual['Q3_Actual']
aiq4plan = ai[['Line_Item', 'Line_Item_ID', 'Q4_Plan', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag', 'Q4_Rolling_Plan']]
aiq4plan['Quarter'] = 'Q4'
aiq4plan['Planned_Actual'] = 'Planned'
aiq4plan['Planned'] = aiq4plan['Q4_Plan']
aiq4actual = ai[['Line_Item', 'Line_Item_ID', 'Q4_Actual', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag', 'Q4_Rolling_Actual']]
aiq4actual['Quarter'] = 'Q4'
aiq4actual['Planned_Actual'] = 'Actual'
aiq4actual['Actual'] = aiq4actual['Q4_Actual']
ai = pd.concat([aiq1plan, aiq1actual, aiq2plan, aiq2actual, aiq3plan, aiq3actual, aiq4plan,
                  aiq4actual], sort=True)
with pd.ExcelWriter('Agile Integration Data Python.xlsx', engine='openpyxl', mode='a') as writer:
    ai.to_excel(writer, sheet_name=str(pd.datetime.now().date()), index=False)
paidsearch = pd.read_excel("~/Downloads/532---Campaign---Cross-Program-Digital.xls")
pd.DataFrame(paidsearch)
paidsearch = paidsearch[['EXPENSES', 'ID', 'Q1 Plan', 'Q2 Plan', 'Q3 Plan', 'Q4 Plan', 'FY Plan', 'Q1 Actual', 'Q2 Actual',
            'Q3 Actual', 'Q4 Actual', 'FY Actual', 'Marketing Activity', 'Marketing Activity Type', 'Event Type',
            'Activity Description']]
paidsearch.columns = ['Line_Item', 'Line_Item_ID', 'Q1_Plan', 'Q2_Plan', 'Q3_Plan', 'Q4_Plan', 'FY_Plan', 'Q1_Actual', 
                'Q2_Actual', 'Q3_Actual', 'Q4_Actual', 'FY_Actual', 'Marketing_Activity', 'Marketing_Activity_Type',
                'Event_Type', 'Activity_Description']
paidsearch['Date_Pulled'] = pd.datetime.now().date()
paidsearch['Function'] = 'GDC Paid Search'
paidsearch['Campaign'] = 'Cloud Native App Dev'
paidsearch['Budget_Name'] = 'GDC Paid Search'
paidsearch['OpenShift_Tag'] = np.where(paidsearch['Activity_Description'].str.contains('openshift|ocp|container|containers',
                                                                           na=False, case=False),'OpenShift', 
                                np.where(paidsearch['Line_Item'].str.contains('openshift|ocp|container|containers',
                                na=False, case=False), 'OpenShift','Middleware'))
paidsearchai = pd.DataFrame.copy(paidsearch)
paidsearchai['Campaign'] = 'Agile Integration'
paidsearch = pd.concat([paidsearch, paidsearchai])
paidsearch['Q1_Plan'] = paidsearch['Q1_Plan']/4
paidsearch['Q1_Actual'] = paidsearch['Q1_Actual']/4
paidsearch['Q2_Plan'] = paidsearch['Q2_Plan']/4
paidsearch['Q2_Actual'] = paidsearch['Q2_Actual']/4
paidsearch['Q3_Plan'] = paidsearch['Q3_Plan']/4
paidsearch['Q3_Actual'] = paidsearch['Q3_Actual']/4
paidsearch['Q4_Plan'] = paidsearch['Q4_Plan']/4
paidsearch['Q4_Actual'] = paidsearch['Q4_Actual']/4
paidsearch['FY_Plan'] = paidsearch['FY_Plan']/4
paidsearch['FY_Actual'] = paidsearch['FY_Actual']/4
paidsearch['Q1_Plan'] = np.where(paidsearch['Q1_Plan'] > 0, paidsearch['Q1_Plan'], 0)
paidsearch['Q1_Actual'] = np.where(paidsearch['Q1_Actual'] > 0, paidsearch['Q1_Actual'], 0)
paidsearch['Q2_Plan'] = np.where(paidsearch['Q2_Plan'] > 0, paidsearch['Q2_Plan'], 0)
paidsearch['Q2_Actual'] = np.where(paidsearch['Q2_Actual'] > 0, paidsearch['Q2_Actual'], 0)
paidsearch['Q3_Plan'] = np.where(paidsearch['Q3_Plan'] > 0, paidsearch['Q3_Plan'], 0)
paidsearch['Q3_Actual'] = np.where(paidsearch['Q3_Actual'] > 0, paidsearch['Q3_Actual'], 0)
paidsearch['Q4_Plan'] = np.where(paidsearch['Q4_Plan'] > 0, paidsearch['Q4_Plan'], 0)
paidsearch['Q4_Actual'] = np.where(paidsearch['Q4_Actual'] > 0, paidsearch['Q4_Actual'], 0)
paidsearch['FY_Plan'] = np.where(paidsearch['FY_Plan'] > 0, paidsearch['FY_Plan'], 0)
paidsearch['FY_Actual'] = np.where(paidsearch['FY_Actual'] > 0, paidsearch['FY_Actual'], 0)
paidsearch['Q2_Rolling_Plan'] = paidsearch['Q1_Plan']+paidsearch['Q2_Plan']
paidsearch['Q2_Rolling_Actual'] = paidsearch['Q1_Actual']+paidsearch['Q2_Actual']
paidsearch['Q3_Rolling_Plan'] = paidsearch['Q2_Rolling_Plan']+paidsearch['Q3_Plan']
paidsearch['Q3_Rolling_Actual'] = paidsearch['Q2_Rolling_Actual']+paidsearch['Q3_Actual']
paidsearch['Q4_Rolling_Plan'] = paidsearch['Q3_Rolling_Plan']+paidsearch['Q4_Plan']
paidsearch['Q4_Rolling_Actual'] = paidsearch['Q3_Rolling_Actual']+paidsearch['Q4_Actual']
paidsearchq1plan = paidsearch[['Line_Item', 'Line_Item_ID', 'Q1_Plan', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag']]
paidsearchq1plan['Quarter'] = 'Q1'
paidsearchq1plan['Planned_Actual'] = 'Planned'
paidsearchq1plan['Planned'] = paidsearchq1plan['Q1_Plan']
paidsearchq1actual = paidsearch[['Line_Item', 'Line_Item_ID', 'Q1_Actual', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag']]
paidsearchq1actual['Quarter'] = 'Q1'
paidsearchq1actual['Planned_Actual'] = 'Actual'
paidsearchq1actual['Actual'] = paidsearchq1actual['Q1_Actual']
paidsearchq2plan = paidsearch[['Line_Item', 'Line_Item_ID', 'Q2_Plan', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag', 'Q2_Rolling_Plan']]
paidsearchq2plan['Quarter'] = 'Q2'
paidsearchq2plan['Planned_Actual'] = 'Planned'
paidsearchq2plan['Planned'] = paidsearchq2plan['Q2_Plan']
paidsearchq2actual = paidsearch[['Line_Item', 'Line_Item_ID', 'Q2_Actual', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag', 'Q2_Rolling_Actual']]
paidsearchq2actual['Quarter'] = 'Q2'
paidsearchq2actual['Planned_Actual'] = 'Actual'
paidsearchq2actual['Actual'] = paidsearchq2actual['Q2_Actual']
paidsearchq3plan = paidsearch[['Line_Item', 'Line_Item_ID', 'Q3_Plan', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag', 'Q3_Rolling_Plan']]
paidsearchq3plan['Quarter'] = 'Q3'
paidsearchq3plan['Planned_Actual'] = 'Planned'
paidsearchq3plan['Planned'] = paidsearchq3plan['Q3_Plan']
paidsearchq3actual = paidsearch[['Line_Item', 'Line_Item_ID', 'Q3_Actual', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag', 'Q3_Rolling_Actual']]
paidsearchq3actual['Quarter'] = 'Q3'
paidsearchq3actual['Planned_Actual'] = 'Actual'
paidsearchq3actual['Actual'] = paidsearchq3actual['Q3_Actual']
paidsearchq4plan = paidsearch[['Line_Item', 'Line_Item_ID', 'Q4_Plan', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag', 'Q4_Rolling_Plan']]
paidsearchq4plan['Quarter'] = 'Q4'
paidsearchq4plan['Planned_Actual'] = 'Planned'
paidsearchq4plan['Planned'] = paidsearchq4plan['Q4_Plan']
paidsearchq4actual = paidsearch[['Line_Item', 'Line_Item_ID', 'Q4_Actual', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag', 'Q4_Rolling_Actual']]
paidsearchq4actual['Quarter'] = 'Q4'
paidsearchq4actual['Planned_Actual'] = 'Actual'
paidsearchq4actual['Actual'] = paidsearchq4actual['Q4_Actual']
paidsearch = pd.concat([paidsearchq1plan, paidsearchq1actual, paidsearchq2plan, paidsearchq2actual, paidsearchq3plan, paidsearchq3actual, paidsearchq4plan,
                  paidsearchq4actual], sort=True)
with pd.ExcelWriter('Paid Search Data Python.xlsx', engine='openpyxl', mode='a') as writer:
    paidsearch.to_excel(writer, sheet_name=str(pd.datetime.now().date()), index=False)
pac = pd.read_excel("~/Downloads/505---Marketing-Communications---Programs---Promotions.xls")
pd.DataFrame(pac)
pac = pac[['EXPENSES', 'ID', 'Q1 Plan', 'Q2 Plan', 'Q3 Plan', 'Q4 Plan', 'FY Plan', 'Q1 Actual', 'Q2 Actual',
            'Q3 Actual', 'Q4 Actual', 'FY Actual', 'Marketing Activity', 'Marketing Activity Type', 'Event Type',
            'Activity Description']]
pac.columns = ['Line_Item', 'Line_Item_ID', 'Q1_Plan', 'Q2_Plan', 'Q3_Plan', 'Q4_Plan', 'FY_Plan', 'Q1_Actual', 
                'Q2_Actual', 'Q3_Actual', 'Q4_Actual', 'FY_Actual', 'Marketing_Activity', 'Marketing_Activity_Type',
                'Event_Type', 'Activity_Description']
pac['Date_Pulled'] = pd.datetime.now().date()
pac['Function'] = 'Portfolio Awareness'
pac['Campaign'] = 'Cloud Native App Dev'
pac['Budget_Name'] = 'Portfolio Awareness'
pac['OpenShift_Tag'] = np.where(pac['Activity_Description'].str.contains('openshift|ocp|container|containers',
                                                                           na=False, case=False),'OpenShift', 
                                np.where(pac['Line_Item'].str.contains('openshift|ocp|container|containers',
                                na=False, case=False), 'OpenShift','Middleware'))
pac = pac[pac['Line_Item'].str.contains('Portfolio Awareness')]
pacai = pd.DataFrame.copy(pac)
pacai['Campaign'] = 'Agile Integration'
pacai['Q1_Plan'] = pacai['Q1_Plan']*.12
pacai['Q1_Actual'] = pacai['Q1_Actual']*.12
pacai['Q2_Plan'] = pacai['Q2_Plan']*.12
pacai['Q2_Actual'] = pacai['Q2_Actual']*.12
pacai['Q3_Plan'] = pacai['Q3_Plan']*.12
pacai['Q3_Actual'] = pacai['Q3_Actual']*.12
pacai['Q4_Plan'] = pacai['Q4_Plan']*.12
pacai['Q4_Actual'] = pacai['Q4_Actual']*.12
pacai['FY_Plan'] = pacai['FY_Plan']*.12
pacai['FY_Actual'] = pacai['FY_Actual']*.12
pac['Q1_Plan'] = pac['Q1_Plan']*.2
pac['Q1_Actual'] = pac['Q1_Actual']*.2
pac['Q2_Plan'] = pac['Q2_Plan']*.2
pac['Q2_Actual'] = pac['Q2_Actual']*.2
pac['Q3_Plan'] = pac['Q3_Plan']*.2
pac['Q3_Actual'] = pac['Q3_Actual']*.2
pac['Q4_Plan'] = pac['Q4_Plan']*.2
pac['Q4_Actual'] = pac['Q4_Actual']*.2
pac['FY_Plan'] = pac['FY_Plan']*.2
pac['FY_Actual'] = pac['FY_Actual']*.2
pac = pd.concat([pac, pacai])
pac['Q1_Plan'] = np.where(pac['Q1_Plan'] > 0, pac['Q1_Plan'], 0)
pac['Q1_Actual'] = np.where(pac['Q1_Actual'] > 0, pac['Q1_Actual'], 0)
pac['Q2_Plan'] = np.where(pac['Q2_Plan'] > 0, pac['Q2_Plan'], 0)
pac['Q2_Actual'] = np.where(pac['Q2_Actual'] > 0, pac['Q2_Actual'], 0)
pac['Q3_Plan'] = np.where(pac['Q3_Plan'] > 0, pac['Q3_Plan'], 0)
pac['Q3_Actual'] = np.where(pac['Q3_Actual'] > 0, pac['Q3_Actual'], 0)
pac['Q4_Plan'] = np.where(pac['Q4_Plan'] > 0, pac['Q4_Plan'], 0)
pac['Q4_Actual'] = np.where(pac['Q4_Actual'] > 0, pac['Q4_Actual'], 0)
pac['FY_Plan'] = np.where(pac['FY_Plan'] > 0, pac['FY_Plan'], 0)
pac['FY_Actual'] = np.where(pac['FY_Actual'] > 0, pac['FY_Actual'], 0)
pac['Q2_Rolling_Plan'] = pac['Q1_Plan']+pac['Q2_Plan']
pac['Q2_Rolling_Actual'] = pac['Q1_Actual']+pac['Q2_Actual']
pac['Q3_Rolling_Plan'] = pac['Q2_Rolling_Plan']+pac['Q3_Plan']
pac['Q3_Rolling_Actual'] = pac['Q2_Rolling_Actual']+pac['Q3_Actual']
pac['Q4_Rolling_Plan'] = pac['Q3_Rolling_Plan']+pac['Q4_Plan']
pac['Q4_Rolling_Actual'] = pac['Q3_Rolling_Actual']+pac['Q4_Actual']
pacq1plan = pac[['Line_Item', 'Line_Item_ID', 'Q1_Plan', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag']]
pacq1plan['Quarter'] = 'Q1'
pacq1plan['Planned_Actual'] = 'Planned'
pacq1plan['Planned'] = pacq1plan['Q1_Plan']
pacq1actual = pac[['Line_Item', 'Line_Item_ID', 'Q1_Actual', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag']]
pacq1actual['Quarter'] = 'Q1'
pacq1actual['Planned_Actual'] = 'Actual'
pacq1actual['Actual'] = pacq1actual['Q1_Actual']
pacq2plan = pac[['Line_Item', 'Line_Item_ID', 'Q2_Plan', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag', 'Q2_Rolling_Plan']]
pacq2plan['Quarter'] = 'Q2'
pacq2plan['Planned_Actual'] = 'Planned'
pacq2plan['Planned'] = pacq2plan['Q2_Plan']
pacq2actual = pac[['Line_Item', 'Line_Item_ID', 'Q2_Actual', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag', 'Q2_Rolling_Actual']]
pacq2actual['Quarter'] = 'Q2'
pacq2actual['Planned_Actual'] = 'Actual'
pacq2actual['Actual'] = pacq2actual['Q2_Actual']
pacq3plan = pac[['Line_Item', 'Line_Item_ID', 'Q3_Plan', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag', 'Q3_Rolling_Plan']]
pacq3plan['Quarter'] = 'Q3'
pacq3plan['Planned_Actual'] = 'Planned'
pacq3plan['Planned'] = pacq3plan['Q3_Plan']
pacq3actual = pac[['Line_Item', 'Line_Item_ID', 'Q3_Actual', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag', 'Q3_Rolling_Actual']]
pacq3actual['Quarter'] = 'Q3'
pacq3actual['Planned_Actual'] = 'Actual'
pacq3actual['Actual'] = pacq3actual['Q3_Actual']
pacq4plan = pac[['Line_Item', 'Line_Item_ID', 'Q4_Plan', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag', 'Q4_Rolling_Plan']]
pacq4plan['Quarter'] = 'Q4'
pacq4plan['Planned_Actual'] = 'Planned'
pacq4plan['Planned'] = pacq4plan['Q4_Plan']
pacq4actual = pac[['Line_Item', 'Line_Item_ID', 'Q4_Actual', 'Marketing_Activity', 'Marketing_Activity_Type',
                   'Event_Type', 'Activity_Description', 'Date_Pulled', 'Function', 'Campaign', 'Budget_Name',
                  'OpenShift_Tag', 'Q4_Rolling_Actual']]
pacq4actual['Quarter'] = 'Q4'
pacq4actual['Planned_Actual'] = 'Actual'
pacq4actual['Actual'] = pacq4actual['Q4_Actual']
pac = pd.concat([pacq1plan, pacq1actual, pacq2plan, pacq2actual, pacq3plan, pacq3actual, pacq4plan,
                  pacq4actual], sort=True)
with pd.ExcelWriter('Portfolio Awareness Data Python.xlsx', engine='openpyxl', mode='a') as writer:
    pac.to_excel(writer, sheet_name=str(pd.datetime.now().date()), index=False)
programs = pd.concat([cnad, ai, paidsearch, pac], sort=True)
with pd.ExcelWriter('Programs Data Python.xlsx', engine='openpyxl', mode='a') as writer:
    programs.to_excel(writer, sheet_name=str(pd.datetime.now().date()), index=False)
alldata = pd.concat([finalregions, programs], sort=True)
with pd.ExcelWriter('All Data Python.xlsx', engine='openpyxl', mode='a') as writer:
    alldata.to_excel(writer, sheet_name=str(pd.datetime.now().date()), index=False)